In [22]:
import mss
import time
import numpy
import cv2
import numpy as np
from ppadb.client import Client
from time import time, sleep
from io import BytesIO
from PIL import Image


client = Client(host="127.0.0.1", port=5037)
# Ruizhu
client.remote_connect("127.0.0.1", 5749)
device = client.device("127.0.0.1:5749")
# Hara Saori
# client.remote_connect("127.0.0.1", 5753)
# device = client.device("127.0.0.1:5753")
# Sharon Roger
# client.remote_connect("127.0.0.1", 5751)
# device = client.device("127.0.0.1:5751")
    
print(device)

cast_img = cv2.imread('cast.png', cv2.IMREAD_UNCHANGED)
cast_img_gray = cv2.cvtColor(cast_img, cv2.COLOR_BGR2GRAY)
w_cast = cast_img.shape[1]
h_cast = cast_img.shape[0]

reel_img = cv2.imread('reel.png', cv2.IMREAD_UNCHANGED)
w_reel = reel_img.shape[1]
h_reel = reel_img.shape[0]

bait_number = 44
clicked_cast = 0
clicked_reel = 0

with mss.mss() as sct:
    # Part of the screen to capture
    monitor_number = 2
    mon = sct.monitors[monitor_number]
    # top left
    monitor = {
        "top": mon["top"] + 33 + 420,  # 100px from the top
        "left": mon["left"] + 0 + 950,  # 100px from the left
        "width": 200,
        "height": 200,
        "mon": monitor_number,
    }
    # bottom left
#     monitor = {
#         "top": mon["top"] + 700 + 420,  # 100px from the top
#         "left": mon["left"] + 0 + 950,  # 100px from the left
#         "width": 200,
#         "height": 200,
#         "mon": monitor_number,
#     }
    # top right
#     monitor = {
#         "top": mon["top"] + 33 + 420,  # 100px from the top
#         "left": mon["left"] + 1280 + 950,  # 100px from the left
#         "width": 200,
#         "height": 200,
#         "mon": monitor_number,
#     }

    while "Screen capturing":
        # Get raw pixels from the screen, save it to a Numpy array
        img = numpy.array(sct.grab(monitor))
#         image = device.screencap()

#         with open('screen.png', 'wb') as f:
#             f.write(image)

#         image = Image.open('screen.png')
#         img = numpy.array(image, dtype=numpy.uint8)
#         img = img[420:950, 620:1150]
        screen_img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        result = cv2.matchTemplate(screen_img_gray, cast_img_gray, cv2.TM_CCOEFF_NORMED)
        yloc, xloc = np.where(result >= .80)
        rectangles = []
        for (x, y) in zip(xloc, yloc):
            rectangles.append([int(x), int(y), int(w_cast), int(h_cast)])
            rectangles.append([int(x), int(y), int(w_cast), int(h_cast)])
        rectangles, weights = cv2.groupRectangles(rectangles, 1, 0.8)
        cv2.imshow("OpenCV/Numpy normal", img)
        for (x, y) in zip(xloc, yloc):
            cv2.rectangle(img, (x, y), (x + w_cast, y + h_cast), (0, 255, 255), 2)
            if(clicked_cast == 0):
                device.shell(f'input tap 1090 530')
                sleep(0.1)
                clicked_cast = 1
                clicked_reel = 0
            cv2.imshow("OpenCV/Numpy normal", img)
            
        reel_result = cv2.matchTemplate(img, reel_img, cv2.TM_CCOEFF_NORMED)
        yloc, xloc = np.where(reel_result >= .70)
        rectangles = []
        for (x, y) in zip(xloc, yloc):
            rectangles.append([int(x), int(y), int(w_reel), int(h_reel)])
            rectangles.append([int(x), int(y), int(w_reel), int(h_reel)])
        rectangles, weights = cv2.groupRectangles(rectangles, 1, 0.8)
        cv2.imshow("OpenCV/Numpy normal", img)
        for (x, y) in zip(xloc, yloc):
            cv2.rectangle(img, (x, y), (x + w_reel, y + h_reel), (0, 255, 255), 2)
            if(clicked_reel == 0):
                device.shell(f'input tap 1090 530')
                sleep(0.1)
                clicked_cast = 0
                clicked_reel = 1
                bait_number -= 1
            cv2.imshow("OpenCV/Numpy normal", img)

        # Display the picture
        cv2.imshow("OpenCV/Numpy normal", img)

        # Display the picture in grayscale
        # cv2.imshow('OpenCV/Numpy grayscale',
        #            cv2.cvtColor(img, cv2.COLOR_BGRA2GRAY))

        # print("fps: {}".format(1 / (time.time() - last_time)))

        # Press "q" to quit
        if (cv2.waitKey(25) & 0xFF == ord("q")) or bait_number == 0:
            cv2.destroyAllWindows()
            break

In [11]:
!pip install cstringio

ERROR: Could not find a version that satisfies the requirement cstringio
ERROR: No matching distribution found for cstringio
